# Convert all texts to sequences of tokens

identified by IDs

In [ ]:
import os, sys, re
import gzip

In [ ]:
import numpy as np

In [ ]:
input_folder = 'data/raw'

In [ ]:
output_folder = 'data/processed'

In [ ]:
import pickle as pkl, gzip, pandas as pd
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+|\'')

In [ ]:
def preprocess(text):
    tokens = tokenizer.tokenize(text)
    preprocessed  = []
    for token in tokens:
        if len(token) > 2 and token == token[0].upper() + token[1:].lower():
            token = "_PROPERNAME_"
        preprocessed.append(token.lower())
    return preprocessed

In [ ]:
import gzip, pickle as pkl
with gzip.GzipFile("abridged_index2word.pkl.gz", "rb") as f:
    abridged_index2word = pkl.load(f)
with gzip.GzipFile("abridged_word2index.pkl.gz", "rb") as f:
    abridged_word2index = pkl.load(f)

In [ ]:
abridged_index2word[:50]

In [ ]:
def preprocess_to_token_id(text):
    tokens = preprocess(text)
    ids = []
    for t in tokens:
        if t in abridged_word2index:
            idx = abridged_word2index[t]
        else: # _UNKNOWN_
            idx = 0
        ids.append(idx)
    return ids

In [ ]:
for root, folder, files in os.walk(input_folder):
    for file_name in files:
        all_token_ids = []
        with open(input_folder + '/' + file_name, 'r') as f:
            all_lines = list(f)
            for l in all_lines:
                token_ids = preprocess_to_token_id(l)
                all_token_ids.extend(token_ids)
        all_token_ids = np.asarray(all_token_ids)
        with gzip.open(output_folder + '/' + file_name + ".gz", 'wb') as f:
            pkl.dump(all_token_ids, f)